In [52]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import f1_score
from sklearn.metrics import fbeta_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import KBinsDiscretizer

import sklearn as skl
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

import seaborn as sns
import matplotlib.pyplot as plt

#loading data

df_sentimentos_data = pd.read_csv('sentiment_analysis.csv') 

#ver colunas

#print(df_sentimentos_data.columns)

#VER SHAPE

df_sentimentos_data.shape

#FORMATO DOS DADOS

df_sentimentos_data.dtypes

df_sentimentos_data.describe()

#MATRIZ DE CORRELAÇÃO = Não vejo muita correlação que justifique retirar colunas

#corr_matrix = df_sentimentos_data.corr(numeric_only=True)
#f, ax = plt.subplots(figsize=(10,14))
#sns.heatmap(corr_matrix, vmin=1, vmax=1,square=True, annot=True)

#VER DISPERSÃO

# First 2x2 grid
#fig, axs = plt.subplots(2, 2, figsize=(12, 10))
#fig.suptitle('Histograms - Set 1')

#sns.histplot(df_sentimentos_data['WebActivity'], ax=axs[0, 0], kde=True)
#sns.histplot(df_sentimentos_data['Sentiment Analysis'], ax=axs[0, 1], kde=True)
#sns.histplot(df_sentimentos_data['SentimentRating'], ax=axs[1, 0], kde=True)
#sns.histplot(df_sentimentos_data['MaritalStatus'], ax=axs[1, 1], kde=True)

# Second 2x2 grid
#fig, axs = plt.subplots(2, 2, figsize=(12, 10))
#fig.suptitle('Histograms - Set 2')

#sns.histplot(df_sentimentos_data['Gender'], ax=axs[0, 0], kde=True)
#sns.histplot(df_sentimentos_data['EstimatedYearlyIncome'], ax=axs[0, 1], kde=True)
#sns.histplot(df_sentimentos_data['NumberOfContracts'], ax=axs[1, 0], kde=True)
#sns.histplot(df_sentimentos_data['Age'], ax=axs[1, 1], kde=True)

# Third 2x2 grid
#fig, axs = plt.subplots(2, 2, figsize=(12, 10))
#fig.suptitle('Histograms - Set 3')

#sns.histplot(df_sentimentos_data['Target'], ax=axs[0, 0], kde=True)
#sns.histplot(df_sentimentos_data['Available401K'], ax=axs[0, 1], kde=True)
#sns.histplot(df_sentimentos_data['CustomerValueSegment'], ax=axs[1, 0], kde=True)
#sns.histplot(df_sentimentos_data['ChurnScore'], ax=axs[1, 1], kde=True)

# Fourth 2x2 grid
#fig, axs = plt.subplots(2, 2, figsize=(12, 10))
#fig.suptitle('Histograms - Set 4')

#sns.histplot(df_sentimentos_data['CallActivity'], ax=axs[0, 0], kde=True)
#sns.histplot(df_sentimentos_data['Products'], ax=axs[0, 1], kde=True)
#sns.histplot(df_sentimentos_data['birthday'], ax=axs[1, 0], kde=True)

#plt.tight_layout()

#a) - VER MISSING VALUES

#print(df_sentimentos_data.isna().sum()) #Não há missing values

#CONTAR NÚMERO DE LINHAS DUPLICADAS

#print(df_sentimentos_data.duplicated().sum()) 

#Numero de linhas duplicadas = 16

#b) - ELIMINAR LINHAS DUPLICADAS

#Eliminamos 16 linhas duplicadas 

df_sentimentos_data.drop_duplicates(inplace=True)

#Eliminamos a coluna do CostumerKey

df_clean = df_sentimentos_data.drop(columns=['CustomerKey'])

#c) - CRIAR UM 'AGE_BINNED' ATTRIBUTE COM 3 BINS DE IGUAL FREQUENCIA NA FEATURE 'AGE'
# aqui estou a considerar caixas com o mesmo número de amostras
estimator = KBinsDiscretizer(n_bins=3, encode='ordinal', strategy='quantile')   #?????????????????mesmo número de elementos em cada bin ou bins com 'escalas iguais' ???????????

df_clean['age_binned'] = estimator.fit_transform(df_clean[['Age']])

# Verificar o resultado - #os bins estão divididos em intervalos de [29,42[ - que contem  4.898 observ   [42,52[ - que contem  4.942 observ [52,100[ - que contem 5.311 
#print('Bin edges')
#print(estimator.bin_edges_[0]) 
#print('Age Groups')
#print(df_clean.groupby(by=['age_binned']).count())

#d) - PARA CADA REGISTO EXTRAIR O ANO, O MES E O DIA DA SEMANA DA FEATURE 'BIRTHDAY'

#print(df_clean['birthday'].dtype) #o birthday está no formato string
df_clean['birthday'] = pd.to_datetime(df_clean['birthday'])#o birthday passou para formato de data

df_clean['year'] = df_clean['birthday'].dt.year
df_clean['month'] = df_clean['birthday'].dt.month
df_clean['day_of_week'] = df_clean['birthday'].dt.day_name()  # Nome completo do dia da semana


#e) - REMOVER USERS QUE TENHAM UMA ATIVIDADE NA PLATADORMA('WEBACTIVITY') DE MENOS DE UMA HORA E QUE TENHAM IDADES SUPERIORES A 70 ANOS
#count_above_70 = (df_clean['Age'] > 70).sum()
#print(count_above_70) #existem 536 pessoas com mais de 70 anos
#count_less_1 = (df_clean['WebActivity'] < 1).sum()
#print(count_less_1) #existem 9148 pessoas com menos de 1 hora de ativade

#?????Remoção dos users que cumprem os dois critérios????
df_clean = df_clean.drop(df_clean.loc[(df_clean['WebActivity']<1) & (df_clean['Age']>70)].index)

#verificação
#count_above_70 = (df_clean['Age'] > 70).sum()
#print(count_above_70) #existem 79 pessoas com mais de 70 anos
#count_less_1 = (df_clean['WebActivity'] < 1).sum()
#print(count_less_1) #existem 8691 pessoas com menos de 1 hora de ativade

#f) - USE 'SENTIMENT ANALYSIS' TO DETERMINE THE MODE OF THE REMAINING ATRIBUTES - NÁO FUNCIONA O QUE TEM NA FICHA???
df_clean = pd.DataFrame(df_clean)

# Agrupar por 'Sentiment Analysis' e calcular a moda dos atributos restantes
result_mode = df_clean.groupby(by=['Sentiment Analysis']).agg(lambda x: pd.Series.mode(x).tolist())

# Exibir o resultado
print(result_mode)

#g) - USE 'SENTIMENT ANALYSIS' AND ‘MARITALSTATUS’, OBTAIN THE AVERAGE(MEAN) OF THE OTHER ATTRIBUTES
print(df_clean.groupby(by=['Sentiment Analysis','MaritalStatus']).mean(numeric_only=True))

                   WebActivity SentimentRating MaritalStatus Gender  \
Sentiment Analysis                                                    
Negative                   [0]             [1]           [M]    [F]   
Positive                   [0]             [4]           [M]    [M]   
Slightly Negative          [0]             [2]           [M]    [M]   
Slightly Positive          [0]             [3]           [M]    [F]   
Very Negative              [0]             [0]           [S]    [M]   
Very Positive              [0]             [5]           [M]    [F]   

                   EstimatedYearlyIncome NumberOfContracts   Age Target  \
Sentiment Analysis                                                        
Negative                         [60000]               [0]  [48]    [1]   
Positive                         [70000]               [2]  [46]    [0]   
Slightly Negative                [60000]               [2]  [53]    [1]   
Slightly Positive                [30000]               [